In [88]:
import pandas as pd
import descartes
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from bokeh.plotting import figure, save, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors, CARTODBPOSITRON

In [89]:
#need to convert to CSV, add header file

# File path
# points1_fp = r"part-00000.csv"
# points2_fp = r"part-00001.csv"

# Read the data
# points = gpd.read_file(points2_fp)

result = pd.read_csv("../data/lat_longs.txt", 
                  sep=' ', 
                  names=["Lat", "Lon", "Wiki URL"])
len(result)

450151

In [90]:
# http://en.wikipedia.org/wiki/Extreme_points_of_the_United_States#Westernmost
top = 49.3457868 # north lat
left = -124.7844079 # west long
right = -66.9513812 # east long
bottom =  24.7433195 # south lat

#def cull(x):
#bottom <= lat <= top and left <= lng <= right
#In US

In [91]:
top = 49.3457868 # north lat
left = -124.7844079 # west long
right = -66.9513812 # east long
bottom =  24.7433195 # south lat
result = result.drop( (result[((bottom <= result.Lat) & (result.Lat <= top)) & ((left <= result.Lon) & (result.Lon <= right)) == False]  ).index)
#result = result.drop(result[(left> result.Lon) & (result.Lon > right)].index)
len(result)

113885

In [92]:
import math
from ast import literal_eval
def getx(x):  
    lat = x[0]
    lon = x[1]
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return x

In [93]:
import math
from ast import literal_eval
def gety(x):  
    lat = x[0]
    lon = x[1]
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return y

In [94]:
result['coords_x'] = result.apply(getx,1)
result['coords_y'] = result.apply(gety,1)
result.head()

,Lat,Lon,Wiki URL,coords_x,coords_y
14,42.034722,-93.620000,"<http://dbpedia.org/resource/Ames,_Iowa>",-1.042173e+07,5.166182e+06
24,46.188889,-123.821111,"<http://dbpedia.org/resource/Astoria,_Oregon>",-1.378370e+07,5.810671e+06
26,30.250000,-97.750000,"<http://dbpedia.org/resource/Austin,_Texas>",-1.088148e+07,3.535726e+06
27,42.281389,-83.748333,"<http://dbpedia.org/resource/Ann_Arbor,_Michigan>",-9.322822e+06,5.203224e+06
28,40.690444,-73.995139,<http://dbpedia.org/resource/Cobble_Hill_Tunnel>,-8.237101e+06,4.966789e+06


In [95]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)
p.circle(x = result['coords_x'],
         y = result['coords_y'])
show(p)